Please follow the rules/instructions for making the assignments. This means, for every exercise:

- Make sure the code is **well-documented**
- Update the README file, found in the same directory as the assignment, this means:
    - Describe the objective of the assignment
    - Provide instructions on how to run your code
    - Include any other relevant notes or dependencies

Also, please be mindful of the rules for the use of Generative AI:
- You may use generative AI tools (e.g., ChatGPT, GitHub Copilot, etc.) in your assignments under the following conditions:
    - Clearly indicate where and how you used generative AI
    - Save and submit the AI conversation alongside your assignment (upload the transcript alongside your assignment)
    - Remember: Using AI tools should support your learning, not replace it. Make sure the work you submit reflects your group’s understanding. All group members are still responsible for understanding and being able to explain all submitted content.

# Exercise 1 - Evi Levels

**a)** When looking at the ESCHER map of the E. coli core model, we do not observe the same pattern as we did for reaction fluxes in a linear pathway. Previously, fluxes in a linear pathway were equal due to mass balance constraints, but for maximal reaction activities, the arrows representing sequential reactions often have different colors or shades, indicating different activity levels. For example, the arrow for PFK has a lighter shade of green than the arrow for FBA, indicating a lower maximal reaction acitivity (in mmol/gDW/h) for PFK compared to FBA. Thus, we can conclude that the maximal reaction activities are not equal to each other in a linear pathway. A reason for this could be that the maximal reaction activities show the estimated maximal activity for each reaction individually, instead of the actual fluxes that are constrained by mass balance. 

**b)** Two examples of reactions that have no maximal reaction activity data are *EX_h2o_e* and *EX_co2_e*, which are H2O exchange and CO2 exchange reactions, respectively. Gene expression-derived data would not be applicable because exchange reactions are not physical reactions, but they are artificial reactions describing the system boundaries. Since there are no genes or enzymes involved, there is no gene expression-derived data associated with them. 

# Exercise 2 - Evi Levels

In [1]:
import cobra
import pandas as pd

In [2]:
# Loading the model
model = cobra.io.load_json_model('e_coli_core.json')

In [3]:
# Taking a look at the model
model

Name,e_coli_core
Memory address,1461ab920
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [4]:
# Loading the gene expression data 
gene_expression_data = pd.read_csv("e_coli_core_expression.csv")
gene_expression_data.head()

,# Reaction ID,reaction activity [mmol/gDW/h]
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12


In [5]:
# renaming the columns for easier usage
gene_expression_data.columns = ['reaction_id', 'max_activity']
gene_expression_data.head()

,reaction_id,max_activity
0,PFK,12.12
1,PFL,1.00
2,PGI,13.12
3,PGK,23.13
4,PGL,8.12


For the ATPM energy maintenance reaction, we leave the lower flux as-is, since it describes cellular energy requirements separate from any maximal reaction activities. 

Then we have to, for reversible reactions, set the lower and upper flux bound to -value and +value, respectively (“value” as the maximal reaction activity). We know a reaction is reversible if the lower bound is < 0 and the upper bound is > 0.

Next we have to, for irreversible reactions, only set the maximal flux bound to value and leave the minimal flux bound at zero. A reaction is irreversible if we have lower bound >= 0 AND upper bound > 0. 

Technically, we could also have another situation for when a reaction is irreversible. This is when we have lower bound < 0 AND upper bound <= 0. However, when looking at the assignment, I suppose we have to ignore this case (at least for now).

We leave the default constraints for reactions without data.

In [6]:
for reaction in model.reactions:
    # Leave ATPM lower flux bound as-is
    if reaction.id == "ATPM":
        continue

    # Get maximal reaction activity
    value = gene_expression_data[gene_expression_data['reaction_id'] == reaction.id]['max_activity']

    # Check if non-empty, so to see if the reaction is in gene_expression_data
    if value.empty:
        # Leave default constraints for reactions without data
        continue
    
    value = value.iloc[0]
    
    # search for reversible reactions
    if reaction.lower_bound < 0 and reaction.upper_bound > 0: 
        # Set lower and upper bound to -value and +value respectively
        reaction.lower_bound = -value
        reaction.upper_bound = value
        
    # search for irreversible reactions
    elif reaction.lower_bound >= 0 and reaction.upper_bound > 0:
        # Leave lower bound at zero
        reaction.lower_bound = 0
        # Set upper bound to value
        reaction.upper_bound = value

Lastly, we take a look at the glucose exchange reaction, *EX_glc__D_e*. We have to remove the maximal absolute flux bound
and use the high absolute default bound instead.

In [7]:
model.reactions.EX_glc__D_e.lower_bound = -1000
model.reactions.EX_glc__D_e.upper_bound = 1000

Printing a table listing each reaction’s lower and upper flux bound:

In [8]:
print(f"{'Reaction ID':<25} | {'Lower bound':<12} | {'Upper bound':<12}\n")
for reaction in model.reactions:
    print(f"{reaction.id:<25} | {reaction.lower_bound:<12} | {reaction.upper_bound:<12}")

Reaction ID               | Lower bound  | Upper bound 

PFK                       | 0            | 12.12       
PFL                       | 0            | 1.0         
PGI                       | -13.12       | 13.12       
PGK                       | -23.13       | 23.13       
PGL                       | 0            | 8.12        
ACALD                     | -1.16        | 1.16        
AKGt2r                    | -3.1         | 3.1         
PGM                       | -20.01       | 20.01       
PIt2r                     | -6.03        | 6.03        
ALCD2x                    | -9.01        | 9.01        
ACALDt                    | -2.29        | 2.29        
ACKr                      | -1.19        | 1.19        
PPC                       | 0            | 2.56        
ACONTa                    | -25.35       | 25.35       
ACONTb                    | -25.35       | 25.35       
ATPM                      | 8.39         | 1000.0      
PPCK                      | 0            | 25.2

# Exercise 3 - Evi Levels

**a)**

FVA for all reactions:

In [9]:
FVA_results = cobra.flux_analysis.variability.flux_variability_analysis(
    model = model, 
    reaction_list = model.reactions, 
    loopless = False, 
    fraction_of_optimum = 0.0
)    

Printing resulting minimal and maximal fluxes per reaction:

In [10]:
print(f"{'Reaction ID':<25} | {'Minimal flux':<25} | {'Maximal flux':<25}\n")
for reaction_id, row in FVA_results.iterrows():
    print(f"{reaction_id:<25} | {row['minimum']:<25} | {row['maximum']:<25}")

Reaction ID               | Minimal flux              | Maximal flux             

PFK                       | 0.334677419354839         | 11.15355552574638        
PFL                       | 0.0                       | 1.0                      
PGI                       | -0.08532258064516078      | 9.020350656890784        
PGK                       | -17.596001612302814       | -0.8793548387096762      
PGL                       | 0.0                       | 4.522387640638967        
ACALD                     | -1.16                     | 0.0                      
AKGt2r                    | -3.1                      | 0.0                      
PGM                       | -17.038932281976653       | -0.879354838709677       
PIt2r                     | 4.781534879440234e-15     | 1.783077286504244        
ALCD2x                    | -1.16                     | -7.657268255399939e-17   
ACALDt                    | -1.16                     | 0.0                      
ACKr           

**b)** We identify reactions with gene expression-imposed maximal reaction activities, whose permissible flux range in forward direction is nonzero yet comes out less than its upper flux bound.

In [11]:
count = 0
# Loop through reactions with gene expression-imposed maximal reaction activities
for reaction_id in gene_expression_data['reaction_id']:
    # ignore FORt
    if reaction_id == 'FORt':
        continue 
        
    # Find permissible flux range in forward direction
    permissible_flux_forward = FVA_results.loc[reaction_id, 'maximum']
    # Get reaction
    reaction = model.reactions.get_by_id(reaction_id)
    # Check if permissible_flux_forward is nonzero (so >0, as it is in forward direction) yet comes out less than its upper flux bound
    if permissible_flux_forward > 0 and permissible_flux_forward < reaction.upper_bound:
        print(reaction_id)
        count +=1

print(f"{count} reactions behave this way.")

PFK
PGI
PGL
PIt2r
ACONTa
ACONTb
PPCK
PDH
ADK1
AKGDH
ATPS4r
PTAr
PYK
CO2t
RPE
CS
CYTBD
ENO
SUCDi
TALA
TKT1
TKT2
TPI
FBA
FUM
G6PDH2r
GAPD
GLCpts
GLUDy
GLUN
GLUSy
GND
ICDHyr
ICL
MALS
O2t
36 reactions behave this way.


Some reactions behave this way because the maximal reaction activities from gene expression are *theoretical* estimates for each reaction individually. In reality, reactions are connected in pathways, and thus the flux through a reaction is constrained by mutliple factors, such as mass balance. As a result, even if a reaction could theoretically carry a higher flux, the constraints can limit the actual permissible flux, so it comes out less than the maximal reaction activity.

**c)**

In [12]:
count = 0
for reaction_id, row in FVA_results.iterrows():
    if row['minimum'] > 0:
        count += 1
        print(reaction_id)
        
print(f"{count} reactions have a positive minimal flux in the FVA.")

PFK
PIt2r
ATPM
ATPS4r
CYTBD
ENO
TPI
EX_h_e
EX_h2o_e
FBA
G6PDH2r
GAPD
GLCpts
ICDHyr
NADH16
O2t
16 reactions have a positive minimal flux in the FVA.


Some reactions have a positive minimal flux in the FVA, which means that some reactions are always happening (in the forward direction), as their flux can never become zero or go in the other direction. This could be bevause some reactions produce metabolites that have a high demand and are used for important following reactions.